### A SageMaker Pipeline

The pipeline that we create follows a typical Machine Learning Application pattern of pre-processing, training, evaluation, and conditional model registration and publication, if the quality of the model is sufficient.

![A typical ML Application pipeline](img/pipeline-full.png)

### Getting some constants

We get some constants from the local execution environment.

In [7]:
import boto3
import sagemaker


region = boto3.Session().region_name
role = sagemaker.get_execution_role()
default_bucket = sagemaker.session.Session().default_bucket()

# Change these to reflect your project/business name or if you want to separate ModelPackageGroup/Pipeline from the rest of your team
model_package_group_name = f"ab3-wip-ml-model-group"
pipeline_name = f"Ab3WipMlModelTrainingPipeline"
print(role)

arn:aws:iam::314997521033:role/service-role/AmazonSageMaker-ExecutionRole-20201209T151511


### Get the pipeline instance

Here we get the pipeline instance from your pipeline module so that we can work with it.

In [9]:
from pipelines.insurance.pipeline import get_pipeline


pipeline = get_pipeline(
    region=region,
    role=role,
    default_bucket=default_bucket,
    model_package_group_name=model_package_group_name,
    pipeline_name=pipeline_name,
)

### Submit the pipeline to SageMaker and start execution

Let's submit our pipeline definition to the workflow service. The role passed in will be used by the workflow service to create all the jobs defined in the steps.

In [10]:
pipeline.upsert(role_arn=role)

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


{'PipelineArn': 'arn:aws:sagemaker:ca-central-1:314997521033:pipeline/ab3wipmlmodeltrainingpipeline',
 'ResponseMetadata': {'RequestId': 'c65ee1d5-96c5-4ad4-9907-ec20d06ded31',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c65ee1d5-96c5-4ad4-9907-ec20d06ded31',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '100',
   'date': 'Sun, 11 Apr 2021 02:54:22 GMT'},
  'RetryAttempts': 0}}

In [ ]:
execution = pipeline.start()

### Pipeline Operations: examining and waiting for pipeline execution

Now we describe execution instance and list the steps in the execution to find out more about the execution.

In [ ]:
execution.describe()

We can wait for the execution by invoking `wait()` on the execution:

In [ ]:
execution.wait()

We can list the execution steps to check out the status and artifacts:

In [ ]:
execution.list_steps()

### Parametrized Executions

We can run additional executions of the pipeline specifying different pipeline parameters. The parameters argument is a dictionary whose names are the parameter names, and whose values are the primitive values to use as overrides of the defaults.

Of particular note, based on the performance of the model, we may want to kick off another pipeline execution, but this time on a compute-optimized instance type and set the model approval status automatically be "Approved". This means that the model package version generated by the `RegisterModel` step will automatically be ready for deployment through CI/CD pipelines, such as with SageMaker Projects.

In [ ]:
execution = pipeline.start(
    parameters=dict(
        ProcessingInstanceType="ml.c5.xlarge",
        ModelApprovalStatus="Approved",
    )
)

In [ ]:
execution.wait()

In [ ]:
execution.list_steps()